In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append(r"C:\Users\wb557957\Desktop\GOSTnets-master")
import GOSTnets as gn

# pip install osmium
import osmium, logging
import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

import time
import psutil
import tracemalloc

In [2]:
pth = r"C:\Users\wb557957\Desktop\LKA"
inPth = r"C:\Users\wb557957\Desktop\LKA\lka_data"
outPth = r"C:\Users\wb557957\Desktop\LKA\lka_outputs"

This script will run the Sri Lanka methodology in the Kandy District (Central region).

due to some issues with combining speeds, unbundling geometries & calculating time using gostnets, I performed these tasks (combSpeed, explode, time_sec) in GIS and brought it back in with those fields. thus skipping the convert_network_to_time function. 

In [132]:
# unbundle geometries - we want to unbundle multipart linestrings. they cannot be read by generateRoadsGDF.
# kandy.roads_raw['geometry']=kandy.roads_raw.geometry.apply(lambda x:gn.unbundle_geometry(x))

In [169]:
kandyshp = gpd.read_file(os.path.join(outPth, "kandy_hwycombspeed_expl_time.shp"))

In [170]:
kandyshp.head()

,stnode,endnode,infra_type,osm_id,mbSpeed,MB,combSpeed,Shape_Leng,ORIG_FID,Shape_Le_1,time_sec,geometry
0,3.543648e+07,2.078305e+09,trunk,5131121.0,1.0,1,1,18.394086,1,18.394086,66.21870,"LINESTRING (459524.932 806174.350, 459508.501 ..."
1,3.543648e+07,3.780934e+09,trunk,23802729.0,4.0,1,4,46.944120,2,46.944120,42.24970,"LINESTRING (459524.932 806174.350, 459567.523 ..."
2,3.543648e+07,4.004621e+09,secondary,23802869.0,8.0,1,8,9.414725,3,9.414725,4.23663,"LINESTRING (459524.932 806174.350, 459527.048 ..."
3,2.078305e+09,3.543648e+07,trunk,5131121.0,1.0,1,1,18.394086,4,18.394086,66.21870,"LINESTRING (459508.501 806166.083, 459524.932 ..."
4,2.078305e+09,3.810950e+09,trunk,5131121.0,4.0,1,4,9.018018,5,9.018018,8.11622,"LINESTRING (459508.501 806166.083, 459500.285 ..."


In [171]:
kandyshp.insert(0, 'New_ID', range(0, 0 + len(kandyshp)))
kandyshp.head()

,New_ID,stnode,endnode,infra_type,osm_id,mbSpeed,MB,combSpeed,Shape_Leng,ORIG_FID,Shape_Le_1,time_sec,geometry
0,0,3.543648e+07,2.078305e+09,trunk,5131121.0,1.0,1,1,18.394086,1,18.394086,66.21870,"LINESTRING (459524.932 806174.350, 459508.501 ..."
1,1,3.543648e+07,3.780934e+09,trunk,23802729.0,4.0,1,4,46.944120,2,46.944120,42.24970,"LINESTRING (459524.932 806174.350, 459567.523 ..."
2,2,3.543648e+07,4.004621e+09,secondary,23802869.0,8.0,1,8,9.414725,3,9.414725,4.23663,"LINESTRING (459524.932 806174.350, 459527.048 ..."
3,3,2.078305e+09,3.543648e+07,trunk,5131121.0,1.0,1,1,18.394086,4,18.394086,66.21870,"LINESTRING (459508.501 806166.083, 459524.932 ..."
4,4,2.078305e+09,3.810950e+09,trunk,5131121.0,4.0,1,4,9.018018,5,9.018018,8.11622,"LINESTRING (459508.501 806166.083, 459500.285 ..."


In [172]:
# set path for shapefile to be read by load osm. 
kandyshp = r"C:\Users\wb557957\Desktop\LKA\lka_outputs\kandy_hwycombspeed_prj_explode.shp"

In [173]:
import GOSTnets as gn
from GOSTnets import load_osm as losm
kandytest = losm.OSM_to_network(kandyshp)

In [174]:
kandytest

In [175]:
# note that compared to other gn workflows, i have already filtered out infra_type of interest (trunk -> unclassified & residential
kandytest.roads_raw

,stnode,endnode,infra_type,osm_id,mbSpeed,MB,combSpeed,Shape_Leng,ORIG_FID,Shape_Le_1,geometry
0,3.543648e+07,2.078305e+09,trunk,5131121.0,1.0,1,1,18.394086,1,0.000167,"LINESTRING (80.63332 7.29317, 80.63317 7.29309)"
1,3.543648e+07,3.780934e+09,trunk,23802729.0,4.0,1,4,46.944120,2,0.000425,"LINESTRING (80.63332 7.29317, 80.63370 7.29335)"
2,3.543648e+07,4.004621e+09,secondary,23802869.0,8.0,1,8,9.414725,3,0.000085,"LINESTRING (80.63332 7.29317, 80.63334 7.29325)"
3,2.078305e+09,3.543648e+07,trunk,5131121.0,1.0,1,1,18.394086,4,0.000167,"LINESTRING (80.63317 7.29309, 80.63332 7.29317)"
4,2.078305e+09,3.810950e+09,trunk,5131121.0,4.0,1,4,9.018018,5,0.000082,"LINESTRING (80.63317 7.29309, 80.63309 7.29306)"
...,...,...,...,...,...,...,...,...,...,...,...
459633,7.403015e+09,7.403015e+09,unclassified,791885094.0,0.0,0,25,34.241869,459630,0.000310,"LINESTRING (80.65493 7.38335, 80.65524 7.38338)"
459634,7.403015e+09,7.403015e+09,unclassified,791885094.0,0.0,0,25,34.241869,459631,0.000310,"LINESTRING (80.65524 7.38338, 80.65493 7.38335)"
459635,7.403015e+09,7.403015e+09,unclassified,791885094.0,0.0,0,25,27.639017,459632,0.000250,"LINESTRING (80.65524 7.38338, 80.65546 7.38349)"
459636,7.403015e+09,7.403015e+09,unclassified,791885094.0,0.0,0,25,27.639017,459633,0.000250,"LINESTRING (80.65546 7.38349, 80.65524 7.38338)"


In [ ]:
# after unbundling geometries, run generate roads gdf
kandytest.generateRoadsGDF()

In [ ]:
# does many things, includign creating a G object weith nodes and edge data
kandytest.initialReadIn()

In [ ]:
gn.save(kandytest.network, 'kandy_unclean_time', os.path.join(pth, 'lka_outputs'))

In [ ]:
# print G that was created in InitialReadIn
G

In [ ]:
gn.example_edge(G)

### [optional] 
#### merge the stats values from walker into the mapbox edges using stnode and endnode. 

these attributes will be useful later in identifying roads for rehab based on slow speeds. 

In [21]:
mb_edges = pd.read_csv('lka_traffic_allEdges.csv')
mb_edges

,Unnamed: 0,stnode,endnode,length,Wkt,infra_type,osm_id,time,mode,geometry,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed,mode_speed,diffMeanMin
0,0,1650104033,6236632580,0.088233,"LINESTRING (80.5488946 5.9451973, 80.548912 5....",trunk,4860427,0.005294,drive,"LINESTRING (80.5488946 5.9451973, 80.548912 5....",1650104033,6236632580,23,37,26.793651,"ModeResult(mode=array([26], dtype=int64), coun...",3.793651
1,1,6236632580,6236632579,0.047336,"LINESTRING (80.548912 5.9451108, 80.5489254 5....",trunk,4860427,0.002840,drive,"LINESTRING (80.548912 5.9451108, 80.5489254 5....",6236632580,6236632579,23,37,26.793651,"ModeResult(mode=array([26], dtype=int64), coun...",3.793651
2,2,6236632579,6236632578,0.056870,"LINESTRING (80.5489254 5.9450654, 80.5489482 5...",trunk,4860427,0.003412,drive,"LINESTRING (80.5489254 5.9450654, 80.5489482 5...",6236632579,6236632578,23,37,26.793651,"ModeResult(mode=array([26], dtype=int64), coun...",3.793651
3,3,6236632575,6236632574,0.050106,"LINESTRING (80.5490246 5.94493, 80.5490642 5.9...",trunk,4860427,0.003006,drive,"LINESTRING (80.5490246 5.94493, 80.5490642 5.9...",6236632575,6236632574,6,35,21.702381,"ModeResult(mode=array([28], dtype=int64), coun...",15.702381
4,4,6236632574,6236632573,0.046789,"LINESTRING (80.5490642 5.9448993, 80.5491031 5...",trunk,4860427,0.002807,drive,"LINESTRING (80.5490642 5.9448993, 80.5491031 5...",6236632574,6236632573,6,35,21.702381,"ModeResult(mode=array([28], dtype=int64), coun...",15.702381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501908,501908,598167932,598167933,0.318076,"LINESTRING (80.8939646 6.8288878, 80.894281699...",secondary,783979650,0.022901,drive,"LINESTRING (80.8939646 6.8288878, 80.894281699...",598167932,598167933,27,27,27.000000,"ModeResult(mode=array([27], dtype=int64), coun...",0.000000
501909,501909,598167933,598167932,0.318076,"LINESTRING (80.8939646 6.8288878, 80.894281699...",secondary,783979650,0.022901,drive,"LINESTRING (80.89428169999999 6.8289127, 80.89...",598167933,598167932,27,27,27.000000,"ModeResult(mode=array([27], dtype=int64), coun...",0.000000
501910,501910,598167933,4137570071,0.143990,"LINESTRING (80.89428169999999 6.8289127, 80.89...",secondary,783979650,0.010367,drive,"LINESTRING (80.89428169999999 6.8289127, 80.89...",598167933,4137570071,27,27,27.000000,"ModeResult(mode=array([27], dtype=int64), coun...",0.000000
501911,501911,4137570071,598167933,0.143990,"LINESTRING (80.89428169999999 6.8289127, 80.89...",secondary,783979650,0.010367,drive,"LINESTRING (80.8944255 6.8289201, 80.894281699...",4137570071,598167933,27,27,27.000000,"ModeResult(mode=array([27], dtype=int64), coun...",0.000000


In [31]:
speed_mb = pd.read_csv('lka_traffic_1_10_variance_wb.csv')
speed_mb

,v1,stnode,endnode,infra_type,length,osm_id,geometry,min_speed,max_speed,mean_speed,mode,mode_count,diffmeanmin,slowroad,range,rangedif
0,0,1650104033,6236632580,trunk,0.000088,4860427,"LINESTRING (80.5488946 5.9451973, 80.548912 5....",23,37,26.793652,26,1786,3.793651,1,14,3.690376
1,1,6236632580,6236632579,trunk,0.000047,4860427,"LINESTRING (80.548912 5.9451108, 80.5489254 5....",23,37,26.793652,26,1786,3.793651,1,14,3.690376
2,2,6236632579,6236632578,trunk,0.000057,4860427,"LINESTRING (80.5489254 5.9450654, 80.5489482 5...",23,37,26.793652,26,1786,3.793651,1,14,3.690376
3,58,35436481,2078304672,trunk,0.000167,5131121,"LINESTRING (80.6333162 7.2931692, 80.6331674 7...",5,15,6.029762,6,1976,1.029762,1,10,9.710982
4,62,2078304672,3810950074,trunk,0.000082,5131121,"LINESTRING (80.6331674 7.2930943, 80.633093 7....",6,10,7.983135,8,1939,1.983135,0,4,2.017008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102607,501728,2508106223,5610108213,primary,0.000174,779949274,"LINESTRING (79.8743238 7.1578648, 79.8741916 7...",40,64,46.771328,51,947,6.771329,0,24,3.544356
102608,501729,5610108213,2508106221,primary,0.000169,779949274,"LINESTRING (79.8741916 7.157751, 79.87406 7.15...",40,64,46.771328,51,947,6.771329,0,24,3.544356
102609,501730,2508106221,2508106220,primary,0.000282,779949274,"LINESTRING (79.87406 7.1576444, 79.8738295 7.1...",40,64,46.771328,51,947,6.771329,0,24,3.544356
102610,501731,2508106220,2508106219,primary,0.000281,779949274,"LINESTRING (79.8738295 7.1574817, 79.8735945 7...",40,64,46.771328,51,947,6.771329,0,24,3.544356


calculating time didn't work in python, so to speed things up i calculated manually in excel and then imported. 

In [ ]:
# adapt gn.convert_network_to_time calculation for time value
speed = 'combSpeed'
length = int('Shape_Leng') * 1000
kph = (length / 1000 / speed)
time_sec = kph * 60 * 60

In [ ]:
# try manually calculating time with the graph and simple numpy
# time will be 
kandy['time']=(kandy['Shape_Leng']/kandy['combSpeed']) * 1000